# Checking the No. of Streams

* Your first step always should be checking how many streams you have in the .xdf file.
* We use the pyxdf library to work with these files.
* The number should be $3$ in our case. If there are three streams then you can continue with other specific checks otherwise the recording was unsuccessful.

In [1]:
import pyxdf
import pandas as pd
import numpy as np

streams, header = pyxdf.load_xdf('moritz_s2.xdf')

for s in streams:
    print(s['info']['name'][0])

ModuleNotFoundError: No module named 'pyxdf'

# EEG Stream Check-Up

* Load the **streams**, change the name of the file and ideal if you put this script in the same folder as the files so you don't have to work through the directory related problems.
* Use `.head()` to check the 1st 5 values and `tail()` for the last 5 values to make sure the device was on during the whole time. The values of all channels should be non-zero.

In [3]:
# Pick the Unicorn/Data stream as EEG
eeg = next(s for s in streams
           if s['info']['name'][0] == 'Unicorn')

# raw data matrix
data = eeg['time_series']

# rebuild timestamps if they’re flat or missing
if (eeg['time_stamps'] is None
    or np.allclose(np.diff(eeg['time_stamps']), 0)):
    try:
        srate = float(eeg['info']['nominal_srate'][0])
    except Exception:
        srate = 250.
    timestamps = np.arange(data.shape[0]) / srate
else:
    timestamps = eeg['time_stamps']

# channel labels (fallback to generic names)
try:
    chan_labels = [
        ch.decode()
        for ch in eeg['info']['desc'][0]
                             ['channels'][0]['channel']
    ]
except Exception:
    chan_labels = [f'ch{i}' for i in range(data.shape[1])]

# build DataFrame
eeg_df = pd.DataFrame(data, columns=chan_labels)
eeg_df.insert(0, 'time', timestamps)

print("EEG DataFrame:", eeg_df.shape)

eeg_df

EEG DataFrame: (411615, 18)


,time,ch0,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,ch9,ch10,ch11,ch12,ch13,ch14,ch15,ch16
0,424264.462008,207969.406250,214526.156250,262586.43750,205574.281250,233423.562500,221402.000000,250628.250000,232598.250000,0.136230,0.981201,0.229492,1.373291,-0.610352,-0.396728,86.666672,160998.0,1.0
1,424264.466004,207962.609375,214518.906250,262583.65625,205569.093750,233417.312500,221397.609375,250623.859375,232593.343750,0.134033,0.981934,0.231934,1.403808,-0.549316,-0.518799,86.666672,160999.0,1.0
2,424264.470000,207975.218750,214530.531250,262595.28125,205583.125000,233431.078125,221411.562500,250639.328125,232607.546875,0.135498,0.982178,0.230957,1.373291,-0.579834,-0.396728,86.666672,161000.0,1.0
3,424264.473996,207993.281250,214549.390625,262599.40625,205602.000000,233447.265625,221423.984375,250653.906250,232626.500000,0.135986,0.980713,0.229004,1.312256,-0.671387,-0.335693,86.666672,161001.0,1.0
4,424264.477992,207972.078125,214526.687500,262570.34375,205579.546875,233422.859375,221396.359375,250626.375000,232600.406250,0.135010,0.979980,0.229492,1.434326,-0.701904,-0.305176,86.666672,161002.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411610,425909.256866,197358.765625,203649.531250,479931.12500,194405.296875,222882.578125,209640.062500,238157.765625,221745.937500,-0.063232,0.868164,0.547852,0.701904,-0.701904,-0.366211,86.666672,572608.0,1.0
411611,425909.260862,197352.406250,203643.984375,479948.40625,194399.296875,222878.187500,209635.859375,238154.546875,221741.296875,-0.061768,0.866943,0.548584,0.732422,-0.640869,-0.366211,86.666672,572609.0,1.0
411612,425909.264858,197348.562500,203643.984375,479958.40625,194395.812500,222880.156250,209640.234375,238160.000000,221743.796875,-0.062256,0.867432,0.548584,0.640869,-0.457764,-0.518799,86.666672,572610.0,1.0
411613,425909.268854,197355.812500,203652.468750,479896.00000,194401.265625,222888.031250,209651.234375,238169.734375,221749.609375,-0.062256,0.867188,0.549072,0.732422,-0.396728,-0.488281,86.666672,572611.0,1.0


# Checking up the Markers Stream

In [4]:
# pick the marker stream by name
marker_streams = [s for s in streams if 'nBack_markers' in s['info']['name'][0]]
if not marker_streams:
    print("No marker stream found containing 'NBackMarkers'")
else:
    mkr = marker_streams[0]

# build a DataFrame of event times + labels
mkr_df = pd.DataFrame({
    'time'  : mkr['time_stamps'],
    'marker': [t[0] for t in mkr['time_series']]
})

# sort and inspect
mkr_df = mkr_df.sort_values('time').reset_index(drop=True)
print(f"Marker events:\n {mkr_df}")

mkr_df


Marker events:
                time                                             marker
0     424282.874180                                     baseline_start
1     424440.539390                                 main_block_0_start
2     424440.539482  sequence_U,E,B,K,X,F,B,F,I,N,D,Y,Z,L,J,G,M,Z,W...
3     424440.539515  targets_12,17,20,22,23,28,30,31,32,35,36,37,40...
4     424440.558291                            main_block_0_trial_0_on
...             ...                                                ...
1493  425797.730251                                           rt_0.522
1494  425797.730259                                       trial_47_end
1495  425797.730268                                   main_block_6_end
1496  425797.730279                         main_block_6_accuracy_0.83
1497  425800.780950                                       trial_17_end

[1498 rows x 2 columns]


,time,marker
0,424282.874180,baseline_start
1,424440.539390,main_block_0_start
2,424440.539482,"sequence_U,E,B,K,X,F,B,F,I,N,D,Y,Z,L,J,G,M,Z,W..."
3,424440.539515,"targets_12,17,20,22,23,28,30,31,32,35,36,37,40..."
4,424440.558291,main_block_0_trial_0_on
...,...,...
1493,425797.730251,rt_0.522
1494,425797.730259,trial_47_end
1495,425797.730268,main_block_6_end
1496,425797.730279,main_block_6_accuracy_0.83


# Checking up the Mic Stream

In [ ]:
# Pick the audio stream by name (e.g., 'MicDecibels')
audio = next(s for s in streams if s['info']['name'][0] == 'MicDecibels')

# Build a DataFrame of audio dB values and timestamps
audio_df = pd.DataFrame({
    'time': audio['time_stamps'],
    'decibels': [row[0] for row in audio['time_series']]
})

# Sort and inspect
audio_df = audio_df.sort_values('time').reset_index(drop=True)

dynamic_range = audio_df['decibels'].max() - audio_df['decibels'].min()
print(f"Audio dB dynamic range: {dynamic_range}")
audio_df

Audio dB dynamic range: 50.367637634277344


(71878, 2)